<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon/blob/main/Maxent_ae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# Numpy 버전 2.0 미만 다운그레이드
# !pip install 'numpy<2.0' -q

In [ ]:
# 필수 패키지 설치
# !pip install earthengine-api -U -q
# !pip install eeSDM -q
# !pip install geemap -U -q
# !pip install pandas seaborn matplotlib -q
# !pip install scikit-learn-extra -q

In [ ]:
# 라이브러리 임포트
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import glob
import ee
import geemap
import eeSDM
import seaborn as sns
import sys
import time

from statsmodels.stats.outliers_influence import variance_inflation_factor
from shapely.geometry import Point
from ipyleaflet import WidgetControl
from ipywidgets import Label

# Earth Engine 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='ee-jjangmo91')

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

#2. 데이터 준비 (AOI, 종 출현, 환경 변수)

In [ ]:
# 연구지역(AOI) 설정
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")
songnisan_park = protected_areas.filter(ee.Filter.eq('WDPAID', 773))
aoi = songnisan_park.geometry().buffer(5000) # 공원 경계 5km까지 완충 지역

# 종 출현(Occurrence) 데이터 설정
base_occurrence_file = '/content/drive/MyDrive/KNPS/Deer/SDM/Data/sdm_occurrences_all_entire_thinned_300m.csv'
df_occurrence = pd.read_csv(base_occurrence_file)
print(f"  - 종 출현 데이터: '{base_occurrence_file}'에서 {len(df_occurrence)}개 좌표 로드 완료")

# 환경 변수 설정
TARGET_SCALE = 30  # 최종 해상도를 30m로 통일
TARGET_CRS = 'EPSG:3857' # GEE 표준 좌표계

# (1, 2, 3) 고도, 경사, 사면향
dem = ee.Image('USGS/SRTMGL1_003')
elevation = dem.select('elevation')
slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)

# (4-8) ESA WorldCover v200 기반 거리 변수 일괄 생성
worldcover = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map')
dist_to_forest = worldcover.eq(10).fastDistanceTransform().sqrt()   # 10: 산림 (Trees)
dist_to_grass = worldcover.eq(30).fastDistanceTransform().sqrt()    # 30: 초지 (Grassland)
dist_to_cropland = worldcover.eq(40).fastDistanceTransform().sqrt() # 40: 경작지 (Cropland)
dist_to_built = worldcover.eq(50).fastDistanceTransform().sqrt()    # 50: 건물밀집지역 (Built-up)
dist_to_water = worldcover.eq(80).fastDistanceTransform().sqrt()    # 80: 수계 (Permanent water bodies)

# 모든 변수를 하나의 이미지로 통합
predictors = ee.Image.cat([
    elevation.rename('elevation'),
    slope.rename('slope'),
    aspect.rename('aspect'),
    dist_to_forest.rename('dist_to_forest'),
    dist_to_grass.rename('dist_to_grass'),
    dist_to_cropland.rename('dist_to_cropland'),
    dist_to_built.rename('dist_to_built'),
    dist_to_water.rename('dist_to_water')
])

# 해상도 및 좌표계 통일 후 AOI에 맞게 자르기
predictors = predictors.setDefaultProjection(crs=TARGET_CRS, scale=TARGET_SCALE).clip(aoi)

print(f"  - 환경 변수: 사용자 정의 8종 구축 완료 (해상도: {TARGET_SCALE}m)")
print(f"  - 구축된 변수: {predictors.bandNames().getInfo()}")

In [ ]:
# 연구 대상 지역(AOI) 설정
aoi = ee.FeatureCollection("WCMC/WDPA/current/polygons") \
        .filter(ee.Filter.eq('WDPAID', 773)) \
        .geometry() \
        .buffer(5000)

# ESA WorldCover 이미지 불러오기
worldcover = ee.ImageCollection('ESA/WorldCover/v200').first().clip(aoi)

# WorldCover 데이터셋의 공식 범례 정의
worldcover_legend_dict = {
    "Trees": "006400", "Shrubland": "ffbb22", "Grassland": "ffff4c",
    "Cropland": "f096ff", "Built-up": "fa0000", "Bare / sparse vegetation": "b4b4b4",
    "Snow and ice": "f0f0f0", "Permanent water bodies": "0064c8",
    "Herbaceous wetland": "0096a0", "Mangroves": "00cf75", "Moss and lichen": "fae6a0",
}

# 시각화 파라미터에 직접 정의한 딕셔너리의 색상 팔레트를 추가합니다.
worldcover_vis_params = {
  "bands": ["Map"],
  "palette": list(worldcover_legend_dict.values())
}

# 지도 생성 및 레이어 추가
m = geemap.Map(center=[36.54, 127.83], zoom=11)
m.add_basemap('HYBRID')

m.addLayer(worldcover, worldcover_vis_params, 'ESA WorldCover 2021')

# legend_dict를 사용
m.add_legend(
    title="ESA WorldCover V200",
    legend_dict=worldcover_legend_dict,
    position='bottomright'
)

# 연구지역 경계선 추가
m.addLayer(ee.Image().paint(aoi, 0, 2), {'palette': 'yellow'}, 'Area of Interest')

display(m)

#3. 다중공선성 분석 및 변수 선택

In [ ]:
!pip install geojson -q

try:
    # CSV의 실제 위도/경도 컬럼명 지정
    LAT_COL = 'latitude'
    LON_COL = 'longitude'

    # 출현 지점의 환경 변수 값 추출
    features = geemap.pandas_to_ee(df_occurrence, latitude_col=LAT_COL, longitude_col=LON_COL)
    sampled_points = predictors.sampleRegions(collection=features, scale=TARGET_SCALE, geometries=False)

    # GEE 결과를 Pandas 데이터프레임으로 변환
    sampled_info = sampled_points.getInfo()
    properties_list = [f['properties'] for f in sampled_info['features']]
    df_predictors = pd.DataFrame(properties_list)[predictors.bandNames().getInfo()].dropna()
    print("환경 값 추출 완료.")

    # 2. VIF 및 상관관계 행렬 계산
    correlation_matrix = df_predictors.corr()
    vif_data = pd.DataFrame()
    vif_data["feature"] = df_predictors.columns
    vif_data["VIF"] = [variance_inflation_factor(df_predictors.values, i) for i in range(len(df_predictors.columns))]

    print("\n[VIF 계산 결과]")
    print(vif_data.to_string(index=False))

    # 3. 결과 시각화
    vif_sorted = vif_data.set_index('feature').sort_values(by='VIF', ascending=False)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    fig.suptitle("Initial Multicollinearity Analysis (8 Variables)", fontsize=18)

    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", ax=ax1, linewidths=.5)
    ax1.set_title('Correlation Matrix', fontsize=14)
    ax1.tick_params(axis='x', rotation=45)

    sns.barplot(x=vif_sorted['VIF'], y=vif_sorted.index, palette='viridis_r', ax=ax2)
    ax2.set_title('Variance Inflation Factor (VIF)', fontsize=14)
    ax2.set_xlabel('VIF Value'); ax2.set_ylabel('')
    ax2.axvline(x=5, color='orange', linestyle='--', label='High Correlation (VIF=5)')
    ax2.axvline(x=10, color='red', linestyle='--', label='Very High Correlation (VIF=10)')
    ax2.legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

except Exception as e:
    print(f"분석 중 오류가 발생했습니다: {e}")

In [ ]:
try:
    # 최종 변수 세트 생성
    bands_to_remove = ['dist_to_forest', 'dist_to_cropland']
    bands_to_keep = predictors.bandNames().removeAll(bands_to_remove)
    predictors_final = predictors.select(bands_to_keep)

    final_predictor_names = predictors_final.bandNames().getInfo()
    print(f"제거된 변수: {bands_to_remove}")
    print(f"최종 선택된 변수 ({len(final_predictor_names)}개): {final_predictor_names}")

    # 다중공선성 재검증
    # 최종 변수 세트(predictors_final)를 사용하여 환경 값을 다시 추출
    features = geemap.pandas_to_ee(df_occurrence, latitude_col='latitude', longitude_col='longitude')
    sampled_points_final = predictors_final.sampleRegions(collection=features, scale=TARGET_SCALE, geometries=False)

    # GEE 결과를 Pandas 데이터프레임으로 변환
    sampled_info_final = sampled_points_final.getInfo()
    properties_list_final = [f['properties'] for f in sampled_info_final['features']]
    df_predictors_final = pd.DataFrame(properties_list_final)[final_predictor_names].dropna()

    # VIF와 상관관계 행렬을 다시 계산
    correlation_matrix_final = df_predictors_final.corr()
    vif_data_final = pd.DataFrame()
    vif_data_final["feature"] = df_predictors_final.columns
    vif_data_final["VIF"] = [variance_inflation_factor(df_predictors_final.values, i) for i in range(len(df_predictors_final.columns))]

    print("\n[최종 VIF 계산 결과]")
    print(vif_data_final.to_string(index=False))

    # 최종 결과 시각화
    vif_sorted_final = vif_data_final.set_index('feature').sort_values(by='VIF', ascending=False)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    fig.suptitle("Final Multicollinearity Analysis (6 Variables)", fontsize=18)

    # 상관관계 히트맵
    sns.heatmap(correlation_matrix_final, annot=True, cmap='coolwarm', fmt=".2f", ax=ax1, linewidths=.5)
    ax1.set_title('Final Correlation Matrix', fontsize=14)
    ax1.tick_params(axis='x', rotation=45)

    # VIF 막대그래프
    sns.barplot(x=vif_sorted_final['VIF'], y=vif_sorted_final.index, palette='viridis_r', ax=ax2)
    ax2.set_title('Final Variance Inflation Factor (VIF)', fontsize=14)
    ax2.set_xlabel('VIF Value'); ax2.set_ylabel('')
    ax2.axvline(x=5, color='orange', linestyle='--', label='Threshold (VIF=5)')
    ax2.legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

except Exception as e:
    print(f"분석 중 오류가 발생했습니다: {e}")

In [ ]:
# 최종 선택된 거리 변수들의 통계치 확인
print(df_predictors_final[['dist_to_grass', 'dist_to_built', 'dist_to_water']].describe())

In [ ]:
# 시각화를 위한 geemap 지도 객체 생성
m = geemap.Map(center=[36.54, 127.83], zoom=11)
m.add_basemap('HYBRID')

# 각 변수별 시각화 파라미터 정의
vis_params = {
    'elevation': {'min': 100, 'max': 1500, 'palette': ['#006633', '#E5FFCC', '#662A00', '#D8D8D8', '#F5F5F5']},
    'slope': {'min': 0, 'max': 60, 'palette': ['#FFFFFF', '#F0E3A2', '#F1BC67', '#ED7E3A', '#E94423']},
    'aspect': {'min': 0, 'max': 360, 'palette': ['#FF0000', '#FFFF00', '#00FF00', '#00FFFF', '#0000FF', '#FF00FF', '#FF0000']},
    # 아래 변수들의 max 값을 더 작게 조정하여 세밀한 변화를 확인
    'dist_to_grass': {'min': 0, 'max': 50, 'palette': ['#E6E6FA', '#C0C0C0', '#808080']},
    'dist_to_built': {'min': 0, 'max': 50, 'palette': ['#FFFFCC', '#FEB24C', '#FD8D3C', '#F03B20', '#BD0026']},
    'dist_to_water': {'min': 0, 'max': 150, 'palette': ['#F7FBFF', '#DEEBF7', '#C6DBEF', '#9ECAE1', '#6BAED6']}
}

# 최종 변수들을 반복문을 통해 지도에 추가
for band in predictors_final.bandNames().getInfo():
    m.addLayer(
        predictors_final.select(band), # 개별 밴드 선택
        vis_params[band],              # 해당 밴드의 시각화 파라미터 적용
        band,                          # 레이어 이름
        True                           # 처음에는 레이어를 켜진 상태로 둠
    )

# 연구지역 경계선 및 범례 추가
m.add_layer(ee.Image().paint(aoi, 0, 2), {'palette': 'yellow'}, 'Area of Interest')
m.add_colorbar_branca(colors=vis_params['elevation']['palette'], vmin=vis_params['elevation']['min'], vmax=vis_params['elevation']['max'],
                      layer_name='elevation')

# 레이어 컨트롤 추가 및 지도 표시
m.add_layer_control()
display(m)

#4. Presence-Background 데이터 생성

In [ ]:
# 모든 예측 변수에 데이터가 있는 유효 영역 마스크 생성
valid_data_mask = predictors_final.reduce(ee.Reducer.allNonZero()).selfMask()

# 유효 영역 내에서만 Background 포인트 생성
print("유효 데이터 영역 내에서 Background 데이터 생성을 시작합니다...")
try:
    background_points = valid_data_mask.stratifiedSample(
        numPoints=5000,
        region=aoi,
        scale=TARGET_SCALE,
        seed=0,
        geometries=True
    )

    # 생성된 background_points의 좌표를 먼저 추출하여 Pandas DataFrame으로 변환
    coords = background_points.geometry().coordinates().getInfo()
    coords_df = pd.DataFrame(coords, columns=['longitude', 'latitude'])

    # 환경 변수 값 추출 (geometries=False는 그대로 유지하여 효율적으로 값만 가져옴)
    background_values = predictors_final.sampleRegions(
        collection=background_points,
        scale=TARGET_SCALE,
        geometries=False
    )

    # .getInfo()를 사용하여 환경 변수 값을 DataFrame으로 가져오기
    background_info = background_values.getInfo()
    properties_list_bg = [f['properties'] for f in background_info['features']]
    env_vars_df = pd.DataFrame(properties_list_bg)

    # 좌표 DataFrame(coords_df)과 환경 변수 DataFrame(env_vars_df)을 합치기
    background_df = pd.concat([coords_df, env_vars_df], axis=1)

    if 'system:index' in background_df.columns:
        background_df = background_df.drop(columns=['system:index'])

    print(f"Background 데이터 추출 및 변환 완료: {len(background_df)}개")

except Exception as e:
    print(f"Background 데이터 처리 중 오류 발생: {e}")
    sys.exit()

# Presence 데이터 환경 값 추출 및 통합
print("\nPresence 데이터 추출을 시작합니다...")
features = geemap.pandas_to_ee(df_occurrence, latitude_col='latitude', longitude_col='longitude')
presence_values = predictors_final.sampleRegions(collection=features, scale=TARGET_SCALE, geometries=False)
presence_info = presence_values.getInfo()
properties_list_pr = [f['properties'] for f in presence_info['features']]
presence_df = pd.DataFrame(properties_list_pr)
print(f"Presence 데이터 추출 완료: {len(presence_df)}개")

# 결측치 제거
background_df.dropna(inplace=True)
presence_df.dropna(inplace=True)

# pa 컬럼 추가 및 병합
background_df['pa'] = 0
presence_df['pa'] = 1
final_modeling_df = pd.concat([presence_df, background_df], ignore_index=True)

# 최종 데이터 확인
print(f"\n최종 Presence-Background 데이터 생성 완료:")
print(f" - 총 데이터 수: {len(final_modeling_df)} 개")
print(f" - Presence (pa=1): {len(final_modeling_df[final_modeling_df['pa']==1])} 개")
print(f" - Background (pa=0): {len(final_modeling_df[final_modeling_df['pa']==0])} 개")
print("\n최종 데이터 샘플:")
display(final_modeling_df.head())

# 최종 모델링 데이터를 드라이브에 저장
SAVE_PATH = '/content/drive/MyDrive/KNPS/Deer/SDM/Data/final_modeling_data_ae.csv'
final_modeling_df.to_csv(SAVE_PATH, index=False)
print(f"\n최종 모델링 데이터가 다음 경로에 저장되었습니다: {SAVE_PATH}")

#5. Spatial Block Corss-Validation

In [ ]:
print("GEE 업로드용 데이터프레임을 정제합니다...")
df_for_gee = final_modeling_df[['longitude', 'latitude', 'pa']].copy()
print("정제 완료.")

# GEE에서 공간 블록 생성
print("\nGEE에서 훈련/테스트용 공간 블록을 생성합니다...")
scale = 2000
grid = aoi.coveringGrid(scale=scale, proj='EPSG:4326')
split = 0.7
training_grid_ee = grid.randomColumn(seed=0).filter(ee.Filter.lt('random', split))
testing_grid_ee = grid.randomColumn(seed=0).filter(ee.Filter.gte('random', split))
print("공간 블록 생성이 완료되었습니다.")

# 정제된 포인트를 GEE FeatureCollection으로 준비
presence_df_for_ee = df_for_gee[df_for_gee['pa'] == 1]
presence_fc = geemap.pandas_to_ee(presence_df_for_ee, latitude_col='latitude', longitude_col='longitude')
presence_fc = presence_fc.map(lambda ft: ft.set('pa', 1))

background_df_for_ee = df_for_gee[df_for_gee['pa'] == 0]
background_fc = geemap.pandas_to_ee(background_df_for_ee, latitude_col='latitude', longitude_col='longitude')
background_fc = background_fc.map(lambda ft: ft.set('pa', 0))

full_fc = presence_fc.merge(background_fc)

# 모든 포인트에 좌표 정보를 속성(property)으로 추가
def add_coords(feat):
    coords = feat.geometry().coordinates()
    return feat.set('longitude', coords.get(0), 'latitude', coords.get(1))

full_fc_with_coords = full_fc.map(add_coords)

# GEE 서버 내에서 공간 분할 및 환경 변수 값 추출
print("\nGEE 서버에서 공간 분할 및 데이터 샘플링을 수행합니다...")
training_data_ee = full_fc_with_coords.filter(ee.Filter.bounds(training_grid_ee))
testing_data_ee = full_fc_with_coords.filter(ee.Filter.bounds(testing_grid_ee))

# sampleRegions의 properties 목록에 'longitude', 'latitude'를 추가하여 최종 결과에 좌표 정보가 포함
training_df = ee.data.computeFeatures({
    'expression': predictors_final.sampleRegions(collection=training_data_ee, properties=['pa', 'longitude', 'latitude'], scale=TARGET_SCALE),
    'fileFormat': 'PANDAS_DATAFRAME'
})

testing_df = ee.data.computeFeatures({
    'expression': predictors_final.sampleRegions(collection=testing_data_ee, properties=['pa', 'longitude', 'latitude'], scale=TARGET_SCALE),
    'fileFormat': 'PANDAS_DATAFRAME'
})

print("GEE 기반 공간 분할 및 데이터 준비가 완료되었습니다.")
print(f" - 훈련 데이터 수: {len(training_df)}")
print(f" - 테스트 데이터 수: {len(testing_df)}")
print("\n생성된 훈련 데이터프레임 샘플:")
display(training_df.head()) # .head()를 통해 좌표 컬럼이 있는지 확인

# 시각화 (기존과 동일)
print("\n분할 결과를 시각화합니다...")
m = geemap.Map(center=[36.54, 127.83], zoom=11)
m.add_basemap('HYBRID')

m.addLayer(training_grid_ee, {'color': 'blue', 'fillColor': 'rgba(0, 0, 255, 0.1)'}, 'Training Blocks')
m.addLayer(testing_grid_ee, {'color': 'red', 'fillColor': 'rgba(255, 0, 0, 0.1)'}, 'Testing Blocks')
m.addLayer(presence_fc, {'color': 'yellow'}, 'Presence Points')

m.add_layer_control()
display(m)

#6. MaxEnt 하이퍼파라미터 튜닝 및 모델 훈련

In [ ]:
from sklearn.metrics import roc_auc_score
import ee
import geemap

print("데이터 준비 및 하이퍼파라미터 튜닝을 시작합니다...")

# 1. 이전 단계에서 생성된 Pandas 데이터프레임(training_df, testing_df)을
#    GEE FeatureCollection으로 변환합니다.
try:
    training_fc = geemap.pandas_to_ee(training_df, latitude_col='latitude', longitude_col='longitude')
    testing_fc = geemap.pandas_to_ee(testing_df, latitude_col='latitude', longitude_col='longitude')
    print("Pandas 데이터프레임을 GEE FeatureCollection으로 변환 완료.")
    print(f" - 훈련 데이터: {training_fc.size().getInfo()} 개")
    print(f" - 테스트 데이터: {testing_fc.size().getInfo()} 개")
except Exception as e:
    print(f"데이터 변환 중 오류 발생: {e}")
    # 이 단계에서 오류가 발생하면, 이전 셀의 training_df와 testing_df가 올바른지 확인해야 합니다.

# 2. 하이퍼파라미터 튜닝
# 'pa' (presence/absence)를 제외한 최종 변수 목록
feature_bands = predictors_final.bandNames().getInfo()

best_auc = -1
best_params = {}
tuning_results = [] # 결과를 저장할 리스트

reg_multipliers = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
auto_feature_options = [True, False]

print("\n하이퍼파라미터 튜닝 루프를 시작합니다...")
for multiplier in reg_multipliers:
    for auto_feat in auto_feature_options:
        try:
            # Maxent 모델 정의
            maxent_model = ee.Classifier.amnhMaxent(
                betaMultiplier=multiplier,
                autoFeature=auto_feat
            )

            # 모델 훈련 (변환된 training_fc 사용)
            trained_model = maxent_model.train(
                features=training_fc,
                classProperty='pa',
                inputProperties=feature_bands
            )

            # 테스트 데이터로 예측 (변환된 testing_fc 사용)
            classified_test = testing_fc.classify(trained_model, 'probability')

            # GEE 결과를 Pandas 데이터프레임으로 가져와 AUC 계산
            predicted_df = geemap.ee_to_df(classified_test)

            if predicted_df is not None and not predicted_df.empty:
                # 'pa' 컬럼의 데이터 타입을 일치시켜줍니다 (예: 정수형)
                true_labels = predicted_df['pa'].astype(int)
                pred_scores = predicted_df['probability']

                # 라벨에 하나의 클래스만 있는지 확인
                if len(true_labels.unique()) < 2:
                    print(f" - 베타 승수: {multiplier}, 자동 특징: {auto_feat}, AUC: 계산 불가 (테스트 데이터에 하나의 클래스만 존재)")
                    continue

                auc_score = roc_auc_score(true_labels, pred_scores)
                result_str = f" - 베타 승수: {multiplier}, 자동 특징: {auto_feat}, AUC: {auc_score:.4f}"
                print(result_str)
                tuning_results.append(result_str)

                # 최고 성능 모델 업데이트
                if auc_score > best_auc:
                    best_auc = auc_score
                    best_params['betaMultiplier'] = multiplier
                    best_params['autoFeature'] = auto_feat

        except Exception as e:
            print(f" - 베타 승수: {multiplier}, 자동 특징: {auto_feat}, 오류 발생: {e}")
            continue

print("\n--- 하이퍼파라미터 튜닝 완료 ---")
if best_params:
    print(f"최적의 하이퍼파라미터: {best_params}")
    print(f"최고 AUC 점수: {best_auc:.4f}")

    # 찾은 최적의 파라미터로 최종 모델 훈련
    print("\n최적의 파라미터로 최종 모델을 훈련합니다...")
    final_model = ee.Classifier.amnhMaxent(**best_params).train(
        features=training_fc,
        classProperty='pa',
        inputProperties=feature_bands
    )
    print("최종 모델 훈련 완료.")
else:
    print("최적의 하이퍼파라미터를 찾지 못했습니다. 오류 메시지를 확인해주세요.")

#7. 모델 적합성 평가

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

# 테스트 데이터에 대한 예측 수행
print("최종 모델을 사용하여 테스트 데이터셋의 예측 확률을 계산합니다...")
try:
    classified_test_final = testing_fc.classify(final_model, 'probability')
    predicted_df = geemap.ee_to_df(classified_test_final)
    predicted_df['pa'] = predicted_df['pa'].astype(int)
    predicted_df['probability'] = predicted_df['probability'].astype(float)
    print("예측 완료.")
except Exception as e:
    print(f"테스트 데이터 예측 중 오류 발생: {e}")

# 필요한 변수들을 미리 정의
true_labels = predicted_df['pa']
pred_scores = predicted_df['probability']


# AUC 점수 계산
print("\n--- AUC 점수 ---")
auc_score = roc_auc_score(true_labels, pred_scores)
print(f"AUC (Area Under the Curve): {auc_score:.4f}")


# 최적의 임계값(Threshold) 및 TSS 찾기
print("\n최적의 임계값을 찾기 위해 TSS를 계산합니다...")
thresholds = np.linspace(0.0, 1.0, 101)
tss_scores = []
sensitivity_scores = []
specificity_scores = []

for t in thresholds:
    pred_labels = (pred_scores >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    tss = sensitivity + specificity - 1
    tss_scores.append(tss)
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

max_tss_index = np.argmax(tss_scores)
optimal_threshold = thresholds[max_tss_index]
max_tss = tss_scores[max_tss_index]

print(f"\n--- 최적 임계값 및 주요 지표 ---")
print(f"TSS를 최대로 만드는 최적 임계값: {optimal_threshold:.4f}")
print(f"최대 TSS (True Skill Statistic): {max_tss:.4f}")
print(f" - 해당 임계값에서의 민감도(Sensitivity): {sensitivity_scores[max_tss_index]:.4f}")
print(f" - 해당 임계값에서의 특이도(Specificity): {specificity_scores[max_tss_index]:.4f}")


# ROC 곡선 및 기타 지표 시각화
print("\n평가 지표를 시각화합니다...")
fpr, tpr, _ = roc_curve(true_labels, pred_scores)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# 첫 번째 그래프: ROC Curve
ax1.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
ax1.set_ylabel('True Positive Rate (Sensitivity)', fontsize=12)
ax1.set_title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
ax1.legend(loc="lower right")
ax1.grid(True)

# 두 번째 그래프: Metrics vs. Threshold
ax2.plot(thresholds, tss_scores, label='TSS', color='blue', lw=2)
ax2.plot(thresholds, sensitivity_scores, label='Sensitivity', color='green', linestyle='--')
ax2.plot(thresholds, specificity_scores, label='Specificity', color='red', linestyle='--')
ax2.axvline(optimal_threshold, color='purple', linestyle=':', label=f'Optimal Threshold ({optimal_threshold:.2f})')
ax2.set_title('Metrics vs. Threshold', fontsize=16)
ax2.set_xlabel('Threshold', fontsize=12)
ax2.set_ylabel('Score', fontsize=12)
ax2.legend()
ax2.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# 훈련 데이터에서 각 변수의 평균값 계산
print("반응 곡선 생성을 위해 변수별 평균값을 계산합니다...")
predictor_names = predictors_final.bandNames().getInfo()
# training_df에는 환경 변수, pa, 좌표가 모두 포함
mean_values = training_df[predictor_names].mean()
print("계산 완료.")

# 그래프를 그릴 Matplotlib subplot 설정
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten() # 2x3 행렬을 1차원 배열로 만듬
fig.suptitle('Response Curves for Predictor Variables', fontsize=20)

# 각 변수에 대해 반복하며 반응 곡선 데이터 생성 및 시각화
print("\n각 변수별 반응 곡선을 생성합니다...")

for i, var_to_plot in enumerate(predictor_names):

    # a. 해당 변수의 값 범위를 100단계로 나눔 (그래프의 x축)
    min_val = training_df[var_to_plot].min()
    max_val = training_df[var_to_plot].max()
    value_range = np.linspace(min_val, max_val, 100)

    # b. GEE에 보낼 테스트 데이터 생성
    features_to_classify = []
    for val in value_range:
        # 평균값 딕셔너리를 복사하여 테스트 케이스 생성
        properties = mean_values.to_dict()
        # 현재 변수의 값만 루프의 값으로 변경
        properties[var_to_plot] = val
        # 지오메트리 없이 속성만 있는 Feature 생성
        features_to_classify.append(ee.Feature(None, properties))

    # c. GEE FeatureCollection으로 변환
    fc_to_classify = ee.FeatureCollection(features_to_classify)

    # d. 최종 모델로 예측 수행
    classified_fc = fc_to_classify.classify(final_model, 'probability')

    # e. 결과(확률 값)를 가져옴 (그래프의 y축)
    predictions = classified_fc.aggregate_array('probability').getInfo()

    # f. 해당 변수에 대한 그래프 그리기
    ax = axes[i]
    ax.plot(value_range, predictions, lw=2)
    ax.set_xlabel(var_to_plot, fontsize=12)
    ax.set_ylabel('Habitat Suitability', fontsize=12)
    ax.set_title(f'Response to {var_to_plot}', fontsize=14)
    ax.grid(True)

# 사용되지 않는 subplot은 숨김
for i in range(len(predictor_names), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
print("\n반응 곡선 생성 완료.")

#8. 예측 지도 생성

In [ ]:
# 최적의 하이퍼파라미터로 최종 모델을 생성
print("훈련된 최종 모델을 사용하여 서식지 적합성 지도를 생성합니다...")
suitability_map = predictors_final.classify(final_model).reproject(crs='EPSG:3857', scale=30)
print("지도 생성 완료.")

# 예측 결과 시각화 준비
# Viridis 색상 팔레트 (낮음: 보라색, 높음: 노란색)
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#440154', '#482677', '#404788', '#33638D',
                '#287D8E', '#1F968B', '#29AF7F', '#55C667',
                '#95D840', '#DCE319']
}

# 지도 객체 생성 및 레이어 추가
print("결과를 시각화합니다...")
m = geemap.Map(center=[36.54, 127.83], zoom=11)
m.add_basemap('HYBRID')

# 서식지 적합성 지도 레이어 추가
m.addLayer(
    suitability_map.select('probability'),
    vis_params,
    'Habitat Suitability'
)
# 지도에 컬러바 범례 추가
m.add_colorbar(
    vis_params,
    label="Habitat Suitability",
    orientation="vertical",
    layer_name="Habitat Suitability"
)

# 속리산 국립공원 경계 레이어 추가
songnisan_park_boundary = ee.Image().paint(songnisan_park, 0, 2) # 경계선만 추출
m.addLayer(
    songnisan_park_boundary,
    {'palette': 'black'},
    'Songnisan Park Boundary'
)

# 출현 지점 레이어 추가
m.addLayer(
    presence_fc,
    {'color': 'red'},
    'Presence Points'
)

# 레이어 컨트롤 추가 및 최종 지도 출력
m.add_layer_control()
display(m)

In [ ]:
# 이전 단계에서 계산된 'optimal_threshold' 변수를 사용
print(f"이전 단계에서 계산된 최적 임계값({optimal_threshold:.4f})을 사용하여 이진 지도를 생성합니다.")

# 이진(Binary) 잠재 분포 지도 생성
potential_distribution_map = suitability_map.select('probability').gt(optimal_threshold)

# 최종 지도 시각화
print("잠재 분포 지도를 시각화합니다...")
binary_vis_params = {'min': 0, 'max': 1, 'palette': ['#D3D3D3', '#006400']} # 비서식지, 서식지

m = geemap.Map(center=[36.54, 127.83], zoom=11)
m.add_basemap('HYBRID')

# 잠재 분포 지도 레이어 추가
m.addLayer(potential_distribution_map, binary_vis_params, 'Potential Distribution (Binary)')

# 국립공원 경계 레이어 추가
park_boundary = ee.Image().paint(songnisan_park, 0, 2)
m.addLayer(park_boundary, {'palette': 'black'}, 'Songnisan Park Boundary')

# 출현 지점 레이어 추가
m.addLayer(presence_fc, {'color': 'red'}, 'Presence Points')

# 범례 추가
m.add_legend(
    title="Distribution",
    legend_dict={"Suitable Habitat": "006400", "Unsuitable Habitat": "D3D3D3"},
    position="bottomright"
)

m.add_layer_control()
display(m)

# 9. 서식지 적합성 5등급 분류 및 면적 산출


In [ ]:
print("\n--- 서식지 적합성 5등급 분류 및 면적 산출을 시작합니다 ---")

# 등급 분류 로직 정의 (확률 0~1을 5등분)
# 1등급: 0.8 < p <= 1.0 (최적 서식지)
# 2등급: 0.6 < p <= 0.8
# 3등급: 0.4 < p <= 0.6
# 4등급: 0.2 < p <= 0.4
# 5등급: 0.0 <= p <= 0.2 (비서식지)

# GEE expression을 사용하여 등급화 이미지 생성
classified_map = suitability_map.expression(
    "(b('probability') > 0.8) ? 1 :" +
    "(b('probability') > 0.6) ? 2 :" +
    "(b('probability') > 0.4) ? 3 :" +
    "(b('probability') > 0.2) ? 4 : 5"
).rename('grade').clip(aoi)

# 픽셀 면적 이미지 생성 (단위: m^2)
pixel_area = ee.Image.pixelArea()

# 등급 이미지와 면적 이미지를 결합
classification_with_area = pixel_area.addBands(classified_map)

# reduceRegion을 통해 등급별 면적 합계 계산
area_stats = classification_with_area.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='grade'
    ),
    geometry=aoi,
    scale=TARGET_SCALE,
    maxPixels=1e13
)

# 결과 데이터프레임 변환 및 정리
stats_list = area_stats.get('groups').getInfo()
df_area = pd.DataFrame(stats_list)

# 면적 단위 변환 및 레이블링
df_area['area_km2'] = df_area['sum'] / 1_000_000  # 1 km^2 = 1,000,000 m^2
df_area['grade'] = df_area['grade'].astype(int)
df_area = df_area.sort_values('grade') # 1등급부터 순서대로 정렬

# 등급 설명 추가
grade_labels = {
    1: 'Grade 1 (0.8-1.0)',
    2: 'Grade 2 (0.6-0.8)',
    3: 'Grade 3 (0.4-0.6)',
    4: 'Grade 4 (0.2-0.4)',
    5: 'Grade 5 (0.0-0.2)'
}
df_area['label'] = df_area['grade'].map(grade_labels)

print("\n[등급별 면적 산출 결과]")
print(df_area[['label', 'area_km2', 'grade']].to_string(index=False))

# 10. 결과 시각화 (면적 그래프 & 등급 지도)

In [ ]:
# 면적 막대 그래프 (Bar Chart)
import matplotlib.pyplot as plt

# 그래프 크기 설정
plt.figure(figsize=(10, 6))

# 데이터 플로팅
bars = plt.bar(df_area['label'], df_area['area_km2'],
               color=['#006400', '#74C476', '#FFFF00', '#FF7F00', '#A9A9A9'])

# 타이틀 및 라벨 설정
plt.title('Area by Habitat Suitability Grade', fontsize=15)
plt.xlabel('Suitability Grade', fontsize=12)
plt.ylabel('Area (km²)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 막대 위에 수치 표시 (소수점 둘째 자리까지)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, height,
             f'{height:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
m = geemap.Map(center=[36.54, 127.83], zoom=11, height='600px', width='1000px')
m.add_basemap('HYBRID')

# 등급별 시각화 파라미터
grade_vis_params = {
    'min': 1,
    'max': 5,
    'palette': ['#006400', '#74C476', '#FFFF00', '#FF7F00', '#A9A9A9']
}

# 레이어 추가 (서식지 등급)
m.addLayer(classified_map, grade_vis_params, 'Suitability Grades (1-5)')

# 레이어 추가 (국립공원 경계)
m.addLayer(ee.Image().paint(songnisan_park, 0, 2), {'palette': 'black'}, 'Park Boundary')

# 범례 추가
legend_dict = {
    'Grade 1 (High Prob)': '#006400',
    'Grade 2': '#74C476',
    'Grade 3': '#FFFF00',
    'Grade 4': '#FF7F00',
    'Grade 5 (Low Prob)': '#A9A9A9'
}
m.add_legend(title="Habitat Grades", legend_dict=legend_dict)

# 레이어 컨트롤 추가
m.add_layer_control()

# 지도 출력
display(m)

# 11. 결과 내보내기

In [ ]:
output_path = '/content/drive/MyDrive/KNPS/Deer/SDM/Data/habitat_suitability_grade_raster_v1.tif'

print(f"파일을 다운로드하여 다음 경로에 직접 저장합니다:\n{output_path}")
print("지도 크기에 따라 시간이 조금 걸릴 수 있습니다 (진행률이 표시됩니다)...")

# 내보내기 수행 (GEE 서버 -> 코랩 -> 드라이브 경로로 저장)
try:
    geemap.ee_export_image(
        classified_map,          # 저장할 이미지
        filename=output_path,    # 전체 경로 (파일명 포함)
        scale=30,                # 해상도
        region=aoi,              # 영역
        crs='EPSG:3857',         # 좌표계
        file_per_band=False      # 밴드가 1개이므로 False
    )
    print("\n저장이 완료되었습니다! 드라이브 폴더를 확인해보세요.")

except Exception as e:
    print(f"\n오류 발생: {e}")

In [ ]:
import os
import geopandas as gpd

# 1. 저장할 경로 설정
output_gpkg_path = '/content/drive/MyDrive/KNPS/Deer/SDM/Data/habitat_suitability_grade_v1.gpkg'

print("1단계: 등급 이미지(Raster)를 벡터(Polygon)로 변환 중입니다...")
# 래스터 -> 벡터 변환
vectors = classified_map.reduceToVectors(
    geometry=aoi,
    crs=classified_map.projection(),
    scale=30,                   # 30m 해상도
    geometryType='polygon',
    eightConnected=False,
    labelProperty='grade',
    reducer=ee.Reducer.countEvery(),
    maxPixels=1e13
)

print("2단계: 데이터를 Python 메모리(GeoDataFrame)로 가져옵니다...")
print("데이터 복잡도에 따라 시간이 조금 걸릴 수 있습니다 (약 1~3분)...")

try:
    # 1. GEE 벡터 데이터를 GeoPandas 데이터프레임으로 변환 (서버 -> 로컬 다운로드)
    gdf = geemap.ee_to_gdf(vectors)

    # 2. GPKG 파일로 저장
    print(f"3단계: GeoPackage 파일로 저장합니다.\n경로: {output_gpkg_path}")
    gdf.to_file(output_gpkg_path, driver='GPKG')

    print("\n저장 완료! 구글 드라이브에서 확인하세요.")
    print("QGIS에서 이 파일을 바로 여실 수 있습니다.")

except Exception as e:
    print(f"\n오류 발생: {e}")
    print("팁: 만약 메모리 부족 오류가 나면, scale을 30에서 50이나 100으로 높여보세요.")